# Version Control Use Case
This example is solely intended as a demonstration to highlight relevant pycelonis functions and properties. In this example, you will learn how to create text-based backups of analyses, knowledge models, package variables, and transformations. More specifically, you will learn:

- How to connect to the EMS
- How to create folders
- How to create backups of all published analyses, knowledge models, package variables and transformations
- Optionally, How to commit the backup folder to GitHub

<blockquote>
    <strong>NOTE:</strong> Any Celonis objects with a <strong>serialized_content</strong> property can be backed up to a YAML or JSON file.
</blockquote>

## Prerequisites
To follow this tutorial, you should have PyCelonis installed and should know how to perform basic interactions with PyCelonis objects. If you don't know how to do this, please complete the **Celonis Basics** tutorial first. Further, it would be helpful to already have the previously mentioned assets inside your EMS. Please refer to the **Studio - Introduction** and **Data Integration - Data Jobs** tutorials for an overview of working with each asset type.

### 1. Import PyCelonis, connect to Celonis API, and create the backup folder

In [1]:
from pycelonis import get_celonis
from datetime import datetime as dt
from pathlib import Path
import json

celonis = get_celonis()
backup_path = Path('IBC Backup')
now = dt.now().strftime("%d-%m-%Y_%H-%M")

if not backup_path.exists():
    backup_path.mkdir()

[2023-01-30 20:53:08,556] INFO: Initial connect successful! PyCelonis Version: 2.0.1


### 2. Create new backup folders for analyses, knowledge models, variables, and transformations

In [2]:
analyses_path = backup_path / f"Analyses_{now}"
kms_path = backup_path / f"KnowledgeModels_{now}"
vars_path = backup_path / f"Variables_{now}"
trans_path = backup_path / f"Transformations_{now}"

print(analyses_path)
print(kms_path)
print(vars_path)
print(trans_path)

analyses_path.mkdir()
kms_path.mkdir()
vars_path.mkdir()
trans_path.mkdir()

IBC Backup/Analyses_30-01-2023_20-53
IBC Backup/KnowledgeModels_30-01-2023_20-53
IBC Backup/Transformations_30-01-2023_20-53
IBC Backup/Variables_30-01-2023_20-53


### 3. Create backups of all published analyses, sorted by workspace

In [ ]:
# Helper Function

def backup_assets(assets, path):
    for asset in assets:

        # skip unpublished assets
        if asset.activated_draft_id is None:
            continue

        file_name = f'{asset.key}.{asset.serialization_type.lower()}'
        file = path / file_name

        file.write_text(asset.serialized_content)

In [4]:
for space in celonis.studio.get_spaces():

    space_folder = f"{space.name}_{space.id}".replace(" ", "_")

    # Create space folders for analyses backup
    space_path = analyses_path / space_folder
    space_path.mkdir()

    # Create backup files
    for pkg in space.get_packages():
        backup_assets(pkg.get_analyses(), space_path)

print("Analyses Backup Complete")

Analyses Backup Complete


### 4. Create backups of all published knowledge models, sorted by workspace

In [5]:
for space in celonis.studio.get_spaces():

    space_folder = f"{space.name}_{space.id}".replace(" ", "_")

    # Create space folders for knowledge models backup
    space_path = kms_path / space_folder
    space_path.mkdir()

    # Create backup files
    for pkg in space.get_packages():
        backup_assets(pkg.get_knowledge_models(), space_path)

print("Knowledge Models Backup Complete")

Knowledge Models Backup Complete


### 5. Create backups of all package variables, sorted by package

In [ ]:
# Helper Function

def backup_variables(variables, package_path):
    for variable in variables:

        file_name = f"{variable.key}.json"
        file = package_path / file_name

        content = json.dumps({
            'key': variable.key,
            'type_': variable.type_,
            'description': variable.description,
            'value': variable.value,
            'package_key': variable.package_key
            })

        file.write_text(content)

In [6]:
for space in celonis.studio.get_spaces():

    space_folder = f"{space.name}_{space.id}".replace(" ", "_")

    # Create space folders for variables backup
    space_path = vars_path / space_folder
    space_path.mkdir()

    # Create backup files
    for pkg in space.get_packages():
        pkg_folder = f"{pkg.name}_{pkg.id}".replace(" ", "_")
        pkg_path = space_path / pkg_folder
        
        variables = pkg.get_variables()
        
        # Skip packages without variables
        if variables == []:
            continue
        
        pkg_path.mkdir()
        backup_variables(variables, pkg_path)

print("Package Variables Backup Complete")

Package Variables Backup Complete


### 6. Create backups of all transformations, sorted by data job*

In [ ]:
# Helper Function

def backup_transformations(transformations, job_path):
    for transformation in transformations:

        # Handle errors retrieving the statement
        try:
            statement = transformation.get_statement()
            # Skip transformations with no statement
            if statement is None:
                continue
        except Exception as e:
            print(f"FAILED to backup: {transformation.name} with id: {transformation.id} \n You either don't have permissions to access the asset or the transformation is proprietary to Celonis.")
            continue

        file_name = f"{transformation.name}.json"
        file = job_path / file_name

        content = json.dumps({
            'id': transformation.id,
            'name': transformation.name,
            'description': transformation.description,
            'statement': statement,
            'pool_id': transformation.pool_id,
            'job_id': transformation.job_id
            })

        file.write_text(content)

In [7]:
for pool in celonis.data_integration.get_data_pools():

    # Create space folders for analyses backup
    pool_folder = f"{pool.name}_{pool.id}".replace(" ", "_")
    pool_path = trans_path / pool_folder
    pool_path.mkdir()

    for job in pool.get_jobs():
        job_folder = f"{job.name}_{job.id}"
        job_path = pool_path / job_folder
        transformations = job.get_transformations()

        # Skip jobs without transformations
        if transformations == []:
            continue

        job_path.mkdir()
        backup_transformations(transformations, job_path)

print("Transformations Backup Complete")

Transformations Backup Complete


**Transformations downloaded from the marketplace are proprietary to Celonis and are unable to backup*

### 7. (Optional) Commit the backup folder to GitHub

Navigate to the backup_path on the command line using the *cd* command, then run:

    git init
    git add .
    git commit -m "Activating version control"

## Conclusion
Congratulations! You have learned how to connect to the EMS, how to create folders, how to create backups of various assets, and how to commit the backup folder to GitHub.